<h1><center>Prediction of power generated by windmill</center></h1>

<center><img src='windmills.jpg'></center>

In this notebook we present you a prediction of power generated by windmill. Dataset we used is from [this](https://www.kaggle.com/datasets/mubashirrahim/wind-power-generation-data-forecasting?fbclid=IwAR0T6vzPk2tEdrWi3-75ipvpJgZDZ3pETaik6PwwSbc2OadeilPo5g9pSdE&select=Location1.csv) kaggle project. Dataset contains meteorological observations and power generated by windmill hourly between years 2017 and 2022. 

In this project we used linear regression as well as SARIMA, AR and Holt-Winters's models to predict generated power. Due to the size of dataset, records were aggregated from hourly to monthly by taking avarage value from every column. After that we used Information Capacity Indices, Graph and Correlation Coefficient Analysis methods to find best variables for our linear model. At the end we used different metrcis to select the best one.

Enjoy! &#x1F600;

Note: You can run notebook directly in your browser by replacing .com with .dev in github link.
Example: [https://github.dev/Nemczek/wind_power_prediction](https://github.dev/Nemczek/wind_power_prediction)

## Linear regression

In [ ]:
# Run only once - installing packages. Needed to run notebook in browser
install.packages(c("lmtest", "orcutt", "car", "Metrics", "performance", "DescTools", "moments"))

### Preprocessing and selecting variables

In [ ]:
library(tidyverse)
library(lmtest)
library(orcutt)
library(car)
library(Metrics)
library(performance)
library(DescTools)
library(moments)
set.seed(7312)

In [ ]:
data <- read.csv("https://raw.githubusercontent.com/Nemczek/wind_power_prediction/main/Location1.csv")
data
summary(data)

In [ ]:
# Aggregating data to monthly by taking avarage value from all days in a month
data$Time <- as.Date(data$Time)
data2 <- data %>%
  mutate(Month = format(Time, "%Y-%m"))

In [ ]:
month_mean <- data2 %>%
  group_by(Month) %>%
  summarise(across(c("temperature_2m", "relativehumidity_2m", "dewpoint_2m", "windspeed_10m", "windspeed_100m", "winddirection_10m", "winddirection_100m", "windgusts_10m", "Power"), mean))
month_mean
mean_data <- month_mean[2:10]

Scatter plots to check for any correlations

In [ ]:
par(mfrow=c(2,4))
plot(mean_data$Power,mean_data$temperature_2m)
plot(mean_data$Power,mean_data$relativehumidity_2m)
plot(mean_data$Power,mean_data$dewpoint_2m)
plot(mean_data$Power,mean_data$windspeed_10m)
plot(mean_data$Power,mean_data$windspeed_100m)
plot(mean_data$Power,mean_data$winddirection_10m)
plot(mean_data$Power,mean_data$winddirection_100m)
plot(mean_data$Power,mean_data$windgusts_10m)

In the plots above we can see a possible strong correlation between Power and windspeed_10m, windspeed_100m and windgusts_10m 

In [ ]:
cat('Corrlation with windspeed_10m', cor(mean_data$Power,mean_data$windspeed_10m), '\n')
cat('Corrlation with windspeed_100m', cor(mean_data$Power,mean_data$windspeed_100m), '\n')
cat('Corrlation with windgusts_10m', cor(mean_data$Power,mean_data$windgusts_10m), '\n')

With correlations that strong, we can expect variables windspeed_10, _100m and windgusts_10m to be good candidates for our model

In [ ]:
# Spliting data into test and train subsets
sample_month <- sample(c(TRUE, FALSE), nrow(mean_data), replace=TRUE, prob=c(0.7,0.3))
train_month  <- mean_data[sample_month, ]
test_month   <- mean_data[!sample_month, ]

In [ ]:
# Calculating stuff for variable selection methods
cor_month <- cor(train_month)
R0_month <- cor_month[1:8,ncol(cor_month)]
R_month <- cor_month[1:8, 1:8]

t <- qt(0.05, nrow(train_month) - 2)
r_star_month <- sqrt(t^2 / (t^2 + nrow(train_month) - 2))
r_star_month

Information Capacity Indices Method was calculated used clearly for convince. Here we're gonna look into two other methods

In [ ]:
# Graph method
R2_m <- R_month
R2_m[which(abs(R2_m) <= r_star_month)] = 0
R2_m[R2_m == 0] = NA
R2_m

With this method we get first model:

$y = a_{0} + a_{1} \cdot windgusts10m + \epsilon$


With our second method there are two ways to get models:
- By using r*
- By taking maximum from minimums from correlations

Both methods give us the same model:

$y = a_{0} + a_{1} \cdot windspeed100m + \epsilon$

Apart from these models we've decided to test another one with both variabels:

$y = a_{0} + a_{1} \cdot windgusts10m + a_{2} \cdot windspeed100m + \epsilon$

### Creating models

In [ ]:
model.month.1 <- lm(Power ~ windgusts_10m, data=train_month)
summary(model.month.1)

Final model: 

$y = -0.37916 + windgusts10m \cdot 0.10099 + \epsilon$

Model explains 71% of the variation and variable is statistically significant as well as F-test p-value. Let's calculate AIC and BIC.

In [ ]:
AIC(model.month.1)
BIC(model.month.1)

We're going to compare this values to other models. Model with lowest AIC and BIC might be the best one. Let's make remaining models the same way.

In [ ]:
model.month.2 <- lm(Power ~ windspeed_100m, data=train_month)
summary(model.month.2)

In [ ]:
AIC(model.month.2)
BIC(model.month.2)

In [ ]:
model.dbl <- lm(Power ~ windspeed_10m + windgusts_10m, data=train_month)
summary(model.dbl)

In [ ]:
AIC(model.dbl)
BIC(model.dbl)

For our last model we removed variables one by one, until only significant remained.

In [ ]:
model_step <- lm(Power ~ . - temperature_2m
                - relativehumidity_2m - dewpoint_2m
                - windspeed_10m - winddirection_10m
                - winddirection_100m - windgusts_10m, data=train_month)
summary(model_step)

### Statistical analysis of residuals

In [ ]:
res_gust <- model.month.1$residuals
res_wind <- model.month.2$residuals
res_dbl <- model.dbl$residual

#### Skewness and curtosis

In [ ]:
skewness(res_gust)
kurtosis(res_gust)

In [ ]:
skewness(res_wind)
kurtosis(res_wind)

In [ ]:
skewness(res_dbl)
kurtosis(res_dbl)

In [ ]:
par(mfrow=c(1,3))
hist(res_gust)
hist(res_wind)
hist(res_dbl)

Skewed left distibutions in all of the cases + leptokurtic

#### Normality 

In [ ]:
shapiro.test(res_gust) 
shapiro.test(res_wind) 
shapiro.test(res_dbl)

First and second model's residuals are distributed normally. Res_wind residuals are close to normality so we can try and continue with them

#### Autocorrelation - Durbin-Watson test

In [ ]:
dwtest(model.month.1, alternative = "two.sided")
dwtest(model.month.2, alternative = "two.sided")
dwtest(model.dbl, alternative = "two.sided")

Residuals in the first model are autocorrelated. To fix this we can try to use Cochran-Orcutt's transformation

In [ ]:
model.co.1 <- cochrane.orcutt(model.month.1)
dwtest(model.co.1, alternative = "two.sided") # Fixed

#### Homoskedasticity - Breusch – Pagan’s test

Checking if residuals are spread equally along regression line on all levels

In [ ]:
bptest(model.co.1)
bptest(model.month.2)
bptest(model.dbl) # Failed

#### Randomness - Harvey – Collier's test

In [ ]:
harvtest(model.co.1)
harvtest(model.month.2)
harvtest(model.dbl)

True realtionship in all models is linear

#### Symmetry

In [ ]:
# model 1
length(model.co.1$residuals[model.co.1$residuals > 0])
n1 <- 44 
m1 <- 22
t1 <- (m1/n1 - 0.5)/sqrt(m1/n1*(1-m1/n1)/(n1-1))
t1

qt(0.975, 43)

# model 2
length(model.month.2$residuals[model.month.2$residuals > 0])
n2 <- 44
m2 <- 20
t2 <- (m2/n2 - 0.5)/sqrt(m2/n2*(1-m2/n2)/(n2-1))
t2

qt(0.975, 43)

# model 3
length(model.dbl$residuals[model.dbl$residuals > 0])
n3 <- 44
m3 <- 22
t3 <- (m3/n3 - 0.5)/sqrt(m3/n3*(1-m3/n3)/(n3-1))
t3 

qt(0.975, 43)

t < qt(0.975, 43) - pass

No model passed all statistical test. Despite that it's worth to continue with analysis, becaue good predictions are all that metters

### Prediction on test data set

In [ ]:
pred.model.co.1 <- predict(model.co.1, newdata=test_month)
pred.model.month.2 <- predict(model.month.2, newdata=test_month)
pred.model.dbl <- predict(model.dbl, newdata=test_month)

true_value <- test_month$Power

plot(true_value, type="l", col="blue")
lines(pred.model.month.2, col="red")
lines(pred.model.co.1, col="green")
lines(pred.model.dbl, col="orange")
legend(x="topleft", legend=c("true_value", "windspeed_100m", "windgusts", "windgusts + windspeed_10m"),
       fill=c("blue", "red", "green", "orange"))

Visually windspeed, windgusts and model with both variables provide good predictions. It's worth to note, that wnidgusts dataset was transfomed. This could result in bad model. To finally choose the best one, let's calculate some errors

#### MAE

In [ ]:
Metrics::mae(true_value, pred.model.co.1)
Metrics::mae(true_value, pred.model.month.2)
Metrics::mae(true_value, pred.model.dbl) # Best

#### MAPE

In [ ]:
Metrics::mape(true_value, pred.model.co.1)
Metrics::mape(true_value, pred.model.month.2) # Best
Metrics::mape(true_value, pred.model.dbl) # Close

#### MSE

In [ ]:
Metrics::mse(true_value, pred.model.co.1)
Metrics::mse(true_value, pred.model.month.2) # Best
Metrics::mse(true_value, pred.model.dbl) # Close

#### RMSE

In [ ]:
Metrics::rmse(true_value, pred.model.co.1)
Metrics::rmse(true_value, pred.model.month.2) # Best
Metrics::rmse(true_value, pred.model.dbl)

In [ ]:
# Let's bring up BIC and AIC too
AIC(model.month.1)
BIC(model.month.1)

In [ ]:
AIC(model.month.2)
BIC(model.month.2)

In [ ]:
AIC(model.dbl)
BIC(model.dbl)

After analysis we can see, that almost all of our models are not bad at predicting Power. Since model windspeed_100m gives the most accurate (but not by far) values, we can assume that's the best one.

## Time series models